In [0]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import math
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import cv2
import copy
##Checking for GPU: 
device = torch.device("cuda")
print(torch.cuda.get_device_name(0))
import seaborn as sns
import pylab
import gzip
import pickle




Tesla K80


In [0]:
from google.colab import drive
import sys

drive.mount('/content/gdrive')
sys.path.append("/content/gdrive/My Drive/Colab Notebooks")
ROOT = "/content/gdrive/My Drive/Colab Notebooks"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    torchvision.transforms.ToPILImage(mode=None),
    torchvision.transforms.Resize((96,96), interpolation=2),
    transforms.ToTensor()
])


##DOWNLOADING VOC DATA SET: DETECTION:
training2 = torchvision.datasets.VOCDetection(ROOT +"/VOC", year='2012', image_set='trainval', download=True, transform=transform_train)
test2 = torchvision.datasets.VOCDetection(ROOT +"/VOC", year='2012', image_set='val', download=True, transform=transform_train)

Using downloaded and verified file: /content/gdrive/My Drive/Colab Notebooks/VOC/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /content/gdrive/My Drive/Colab Notebooks/VOC/VOCtrainval_11-May-2012.tar


In [0]:
##TRAINING DATA SET: 
images_vehicle = torch.zeros(3542,3,96,96)
labels_name = []
bound_box_array = []
bound_box_temp  = [] 
j = 0

##Extracting Vehicles from the VOC dataset
for i in range(len(training2)):
  image, image_label = training2[i]
  if (type(image_label['annotation']['object']) is list):
      check = image_label['annotation']['object'][0]['name']
      if (check == 'bicycle' or check == 'boat' or check == 'bus' or check == 'car' or check == 'motorbike' or check == 'train' or check == 'aeroplane'):
        bound_box_temp.append(int(image_label['annotation']['object'][0]['bndbox']['xmin']))
        bound_box_temp.append(int(image_label['annotation']['object'][0]['bndbox']['xmax']))
        bound_box_temp.append(int(image_label['annotation']['object'][0]['bndbox']['ymin']))
        bound_box_temp.append(int(image_label['annotation']['object'][0]['bndbox']['ymax']))
        bound_box_array.append(bound_box_temp)
        bound_box_temp = [] 
        labels_name.append(check)
        images_vehicle[j] = image
        j = j + 1
  elif(type(image_label['annotation']['object']) is dict):
      check2 = image_label['annotation']['object']['name']
      if (check2 == 'bicycle' or check2 == 'boat' or check2 == 'bus' or check2 == 'car' or check2 == 'motorbike' or check2 == 'train' or check2 == 'aeroplane'):
        bound_box_temp.append(int(image_label['annotation']['object']['bndbox']['xmin']))
        bound_box_temp.append(int(image_label['annotation']['object']['bndbox']['xmax']))
        bound_box_temp.append(int(image_label['annotation']['object']['bndbox']['ymin']))
        bound_box_temp.append(int(image_label['annotation']['object']['bndbox']['ymax']))
        bound_box_array.append(bound_box_temp)
        bound_box_temp = [] 
        labels_name.append(check2)
        images_vehicle[j] = image
        j = j + 1




In [0]:
##TESTING DATA SET: 
images_vehicle_TEST = torch.zeros(1766,3,96,96)
labels_name_test = []
bound_box_array_test = []
bound_box_temp_test  = [] 
j = 0

##Extracting Vehicles from the VOC dataset
for i in range(len(test2)):
  image, image_label = test2[i]
  if (type(image_label['annotation']['object']) is list):
      check = image_label['annotation']['object'][0]['name']
      if (check == 'bicycle' or check == 'boat' or check == 'bus' or check == 'car' or check == 'motorbike' or check == 'train' or check == 'aeroplane'):
        bound_box_temp_test.append(int(image_label['annotation']['object'][0]['bndbox']['xmin']))
        bound_box_temp_test.append(int(image_label['annotation']['object'][0]['bndbox']['xmax']))
        bound_box_temp_test.append(int(image_label['annotation']['object'][0]['bndbox']['ymin']))
        bound_box_temp_test.append(int(image_label['annotation']['object'][0]['bndbox']['ymax']))
        bound_box_array_test.append(bound_box_temp_test)
        bound_box_temp_test = [] 
        labels_name_test.append(check)
        images_vehicle_TEST[j] = image
        j = j + 1
  elif(type(image_label['annotation']['object']) is dict):
      check2 = image_label['annotation']['object']['name']
      if (check2 == 'bicycle' or check2 == 'boat' or check2 == 'bus' or check2 == 'car' or check2 == 'motorbike' or check2 == 'train' or check2 == 'aeroplane'):
        bound_box_temp_test.append(int(image_label['annotation']['object']['bndbox']['xmin']))
        bound_box_temp_test.append(int(image_label['annotation']['object']['bndbox']['xmax']))
        bound_box_temp_test.append(int(image_label['annotation']['object']['bndbox']['ymin']))
        bound_box_temp_test.append(int(image_label['annotation']['object']['bndbox']['ymax']))
        bound_box_array_test.append(bound_box_temp)
        bound_box_temp_test = [] 
        labels_name_test.append(check2)
        images_vehicle_TEST[j] = image
        j = j + 1




**Convert labels name to number** 

In [0]:
def name2num(labels_name):
  label_num2 = []
  
  for i in range(len(labels_name)):
    if(labels_name[i] == 'car'):
      label_num2.append(0)
    elif(labels_name[i] == 'aeroplane'):
      label_num2.append(1)
    elif(labels_name[i] == 'bicycle'):
      label_num2.append(2)
    elif(labels_name[i] == 'train'):
      label_num2.append(3)
    elif(labels_name[i] == 'bus'):
      label_num2.append(4)
    elif(labels_name[i] == 'motorbike'):
      label_num2.append(5)
    elif(labels_name[i] == 'boat'):
      label_num2.append(6)    
  return label_num2

In [0]:
label_num = name2num(labels_name)
label_num2 = name2num(labels_name_test)

In [0]:


def bbox_to_rect(bbox, color):

    return plt.Rectangle(
        xy=(bbox[0], bbox[2]), width=bbox[1]-bbox[0], height=bbox[3]-bbox[2],
        fill=False, edgecolor=color, linewidth=2)
  

class VOC(Dataset):
    def __init__(self,trn_val_tst = 0, transform=None):
        if trn_val_tst == 0:
            #trainloader
            self.images = images_vehicle
            self.labels = label_num
        else:
            #testloader
            self.images = images_vehicle_TEST
            self.labels = label_num2
            
        self.images = np.float32(self.images)/1.0
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
   
        sample = self.images[idx,:]
        labels = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, labels
      
      
transform_train = transforms.Compose([
    transforms.ToTensor(),
    torchvision.transforms.ToPILImage(mode=None),
    transforms.RandomResizedCrop(32),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])      
      
      
train_set = VOC(trn_val_tst=0, transform = None)
train_set_augment = VOC(trn_val_tst=0, transform = transform_train) 
test_set = VOC(trn_val_tst=1, transform = None)


train_loader = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=100, 
                                           shuffle=True)

train_loader_real = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=100, 
                                           shuffle=True)

train_loader_aug = torch.utils.data.DataLoader(dataset=train_set_augment, 
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_set, 
                                           batch_size=100, 
                                           shuffle=False)



  

**Confusion Matrix**

In [0]:
def confusion(actual,assign):
  confusion_arr = torch.zeros(7,7)
  for j in range(len(actual)):
    for i in range(len(actual[j])):
      confusion_arr[actual[j][i]][assign[j][i]] += 1
    
  return confusion_arr

**Model Clarification**

In [0]:
# Model for training
class DeepLearning(nn.Module):
    def __init__(self):
        # Call the __init__ function of the parent nn.module class
        super(DeepLearning, self).__init__()
        
        
        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32,64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),

            # Conv Layer block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2)
        )
        
        ##GAP Layer: 
        self.GAP = nn.AdaptiveAvgPool2d(1)
        
        ##Fully Connected Layers: 
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(256, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(512, 7)
        )
        
    ## Performing Foward Propogation: 
    def forward(self, x):
        x = self.conv_layer(x)
        x = self.GAP(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x 

In [0]:
# Model for training
class ShallowLearning(nn.Module):
    def __init__(self):
        # Call the __init__ function of the parent nn.module class
        super(ShallowLearning, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=0),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0),
            nn.ReLU())
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=0),
            nn.ReLU())
        self.maxpool =  nn.MaxPool2d(kernel_size=3, stride = 3)
        self.fc1 = nn.Sequential(
            nn.Linear(1152, 128),
            nn.ReLU())
        self.fc2 = nn.Linear(128,7)
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.maxpool(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out 

**Training and Validating Data**

In [0]:
# model = ShallowLearning()
model = DeepLearning()
# model = torchvision.models.resnet101(pretrained=False, progress=True)

In [0]:
##Defining Step Size:  
total_step = len(train_loader)
total_step_val = len(test_loader)
softmax = nn.Softmax(dim=1)


num_epochs = 60
learning_rate = 0.0001


model.cuda()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
loss_list = []
loss2_list = []
acc_list = []
acc2_list = []
acc3_list = [] 


for epoch in range(num_epochs):
    total = 0
    correct = 0
    model.train()
    #Train the neural network using Augmented Data Set: 
    if (epoch%2==0):
      train_loader = train_loader_aug
    else:
      ##Train the neural network using the Original Data Set: 
      train_loader = train_loader
    for i, (images, labels) in enumerate(train_loader_real):
          
    ##Training 
        images = images.to(device)
        labels = labels.long()
        labels = labels.to(device)
   
            
        # Run the forward pass (Training):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
       

        
        # Backprop and perform Adam optimisation
        loss.backward()
        optimizer.step()
        
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted== labels.long()).sum().item()
       
        if (i+1) % total_step == 0:
            loss_list.append(loss.item())
            acc_list.append(correct / total)
            print('Epoch [ {}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
          
        if (i+1) % total_step == 0:  
            model.eval()
            with torch.no_grad():
              correct_val = 0
              total_val = 0
              for j, (images_valid, labels_valid) in enumerate(test_loader):
                  images_valid = images_valid.to(device)
                  labels_valid = labels_valid.to(device)
                  outputs2 = model(images_valid)
                  loss2 = criterion(outputs2, labels_valid.long())
                  _, predicted_valid = torch.max(outputs2.data, 1)
                  total_val += labels_valid.size(0)
                  correct_val += (predicted_valid == labels_valid.long()).sum().item()
               
                  if (j+1) % total_step_val == 0:
                    loss2_list.append(loss2.item())
                    acc2_list.append(correct_val / total_val)
                    print('Epoch [ {}/{}], Step_val [{}/{}], Loss_val: {:.4f}, Accuracy_val: {:.2f}%'
                  .format(epoch + 1, num_epochs, j + 1, total_step_val, loss2.item(),
                          (correct_val / total_val) * 100))
                    

Epoch [ 1/60], Step [36/36], Loss: 1.5894, Accuracy: 28.66%
Epoch [ 1/60], Step_val [18/18], Loss_val: 2.0806, Accuracy_val: 18.46%
Epoch [ 2/60], Step [36/36], Loss: 1.7574, Accuracy: 32.41%
Epoch [ 2/60], Step_val [18/18], Loss_val: 1.5861, Accuracy_val: 34.48%
Epoch [ 3/60], Step [36/36], Loss: 1.4294, Accuracy: 35.88%
Epoch [ 3/60], Step_val [18/18], Loss_val: 1.5718, Accuracy_val: 35.05%
Epoch [ 4/60], Step [36/36], Loss: 1.5840, Accuracy: 38.26%
Epoch [ 4/60], Step_val [18/18], Loss_val: 1.5911, Accuracy_val: 41.90%
Epoch [ 5/60], Step [36/36], Loss: 1.6568, Accuracy: 41.36%
Epoch [ 5/60], Step_val [18/18], Loss_val: 1.4750, Accuracy_val: 44.51%
Epoch [ 6/60], Step [36/36], Loss: 1.5350, Accuracy: 43.79%
Epoch [ 6/60], Step_val [18/18], Loss_val: 1.3880, Accuracy_val: 47.45%
Epoch [ 7/60], Step [36/36], Loss: 1.5471, Accuracy: 46.10%
Epoch [ 7/60], Step_val [18/18], Loss_val: 1.5074, Accuracy_val: 44.68%
Epoch [ 8/60], Step [36/36], Loss: 1.3347, Accuracy: 49.44%
Epoch [ 8/60], S

In [0]:
#Plotting loss vs epoch for training and validation:
loss_list_plot= [] 
loss_list2_plot = []
epoch_plot = []
for i in range(len(loss_list)):
  if(i%2!=0):
    loss_list_plot.append(loss_list[i])
    loss_list2_plot.append(loss2_list[i])
    epoch_plot.append(i+2)


fig, ax = plt.subplots()
  
ax.plot(epoch_plot, loss_list_plot, '-b', label='Training' )
ax.plot(epoch_plot, loss_list2_plot, '--r', label='Validation')
plt.title("Loss Vs Epoch")
plt.xlabel("Number of Epoch")
plt.ylabel("Loss")
leg = ax.legend();

In [0]:
#Plotting Accuracy vs Epoch for both valdiation and training 

acc_list_plot= [] 
acc_list2_plot = []

for i in range(len(acc_list)):
  if(i%2!=0):
    acc_list_plot.append(acc_list[i])
    accc_list2_plot.append(acc2_list[i])

fig, ax = plt.subplots()

ax.plot(epoch_plot, acc_list_plot,'-b', label='Training' )
ax.plot(epoch_plot, acc_list2_plot,  '-k', label='Test')


plt.title("Accuracy Vs Epoch")
plt.xlabel("Number of Epoch")
plt.ylabel("Accuracy")
leg = ax.legend();

In [0]:
predicted_train_real = []
labels_train_real = []

predicted_test_real = []
labels_test_real = []
score_max = [] 

##Training: 
model.eval()
with torch.no_grad():
    correct_train = 0
    total_train = 0
    for j, (images_train, labels_train) in enumerate(train_loader):
            images_train = images_train.to(device)
            labels_train = labels_train.to(device)
            outputs = model(images_train)
            _, predicted_train = torch.max(outputs.data, 1)
            total_train += labels_train.size(0)
            correct_train += (predicted_train == labels_train.long()).sum().item()
            predicted_train_real.append(predicted_train)
            labels_train_real.append(labels_train)
            
print('Training Accuracy of the model on the 3542 Training images: {} %'.format((correct_train / total_train) * 100))

##Testing: 
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for j, (images_test, labels_test) in enumerate(test_loader):
            images_test = images_test.to(device)
            labels_test = labels_test.to(device)
            outputs2 = model(images_test)
            m = torch.nn.Softmax(dim=1)
            _, predicted_test = torch.max(outputs2.data, 1)
            total += labels_test.size(0)
            correct += (predicted_test == labels_test.long()).sum().item()
            predicted_test_real.append(predicted_test)
            labels_test_real.append(labels_test)
            score_max.append(torch.max(m(outputs2.data), 1))
            
            

    print('Test Accuracy of the model on the 1776 Test images: {} %'.format((correct / total) * 100))
    predicted_test_real = torch.cat(predicted_test_real).long()
    labels_test_real = torch.cat(labels_test_real).long()
    score_max_real = []
  
    for i in range(18):
      score_max_real.append(score_max[i][0])
    
score_max_real = torch.cat(score_max_real)




In [0]:
print('confusion matrix of testing images')
print(confusion(labels_2000, predicted_2000))

print('confusion matrix of training images')
print(confusion(labels_train_8000, predicted_train_8000))


In [0]:
from google.colab import files
uploaded = files.upload()

import os
os.getcwd()


In [0]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread("images.jpg")
img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_cvt)
plt.show()
img_cvt = torch.Tensor(img_cvt).unsqueeze(0)

      
transform_train = transforms.Compose([
    transforms.ToTensor(),
    torchvision.transforms.ToPILImage(mode=None),
    torchvision.transforms.Resize((96,96), interpolation=2),
    transforms.ToTensor()
])
img_3d = transform_train(img)
print(img_3d.shape)
img = torch.reshape(torch.Tensor(img_3d),[1,3,96,96]).to(device)

test = model(img)
_, predicted_valid = torch.max(test.data, 1)
prob = (torch.max(m(test.data), 1))
print(prob[0])

if (predicted_valid == 0):
  print("Car")
elif(predicted_valid == 1):
  print("aeroplane")
elif(predicted_valid == 2):
  print("Bicycle")
elif(predicted_valid == 3):
  print("Train")
elif(predicted_valid == 4):
  print("Bus")
elif(predicted_valid == 5):
  print("Motorbike")
elif(predicted_valid == 6):
  print("Boat")


In [0]:
from matplotlib.patches import Rectangle
def Occlusion_exp(image, occluding_size, occluding_pixel, occluding_stride):
    _, height, width = image.shape
    print(height, width)
    output_height = int(math.ceil((height-occluding_size) / occluding_stride + 1))
    output_width = int(math.ceil((width-occluding_size) / occluding_stride + 1))
    heatmap = np.zeros((output_height, output_width))
    start_rec_h = []
    for h in range(output_height):
        for w in range(output_width):
            # Occluder region:
            h_start = h * occluding_stride
            w_start = w * occluding_stride
            h_end = min(height, h_start + occluding_size)
            w_end = min(width, w_start + occluding_size)
            # Getting the image copy, applying the occluding window and classifying it again:
            input_image = copy.copy(image)
            input_image[:,h_start:h_end, w_start:w_end] =  occluding_pixel
            
            im = np.expand_dims(input_image, axis=0)
            model.eval()
            with torch.no_grad():
              out = model(torch.from_numpy(im).float().to(device))
              prob = (torch.max(m(out.data),1))
              prob = (prob.cpu().data.numpy().item())
              print(prob)
              heatmap[h,w] = prob
              if prob>0.6 & (not start_rec_h):
                start_rec_w = w_start
                start_rec_h = h_start
                
    plt.gca().add_patch(Rectangle((start_rec_h,start_rec_w),40,30,linewidth=2,edgecolor='g',facecolor='none'))       
    return heatmap

In [4]:

img = cv2.imread("images.jpg")
img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_cvt)

heatmap = Occlusion_exp(img_3d,3,2,1)

NameError: ignored